In [1]:
import pandas as pd

# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
movie_path = '/Users/kimseohyun/Desktop/패스트캠퍼스/프로젝트/MLProject2_영화_추천_시스템/code/ml-project-ml-pjt-2/src/datasets/movie.csv'
rating_path = '/Users/kimseohyun/Desktop/패스트캠퍼스/프로젝트/MLProject2_영화_추천_시스템/code/ml-project-ml-pjt-2/src/datasets/rating.csv'
rating_dt = pd.read_csv(rating_path)  
movie_dt = pd.read_csv(movie_path)

In [2]:
#결측치가 32000개 이상인건 의미없다고 판단 (영화가 4만 5천개인데 3만5천개면 )75% 결측)
drop_column_list = list(movie_dt.columns[movie_dt.isnull().sum()<=35000])
movie_dt = movie_dt[drop_column_list]

#status가 released가 아닌 영화들은 볼수 없으니 제외
#데이터셋이 과거의 자료여서 post production, In Production 등 production중인 영화들은 이미 개봉했을 수 있지만
#무비 데이터를 최신으로 변경하기 전까지는 현재 데이터셋에선 우선 제거하는걸로
movie_dt = movie_dt[movie_dt['status'] == 'Released'].dropna(subset=['status'])

#지금단계에서 필요없어보이는 column 제거
movie_dt = movie_dt.drop(['overview', 'poster_path', 'tagline', 'spoken_languages'], axis=1)

In [3]:
print(movie_dt['id'].dtype)
print(rating_dt['tmdbId'].dtype)

int64
int64


In [4]:
# 'id' 열을 정수형으로 변환
movie_dt['id'] = movie_dt['id'].astype(int)
rating_dt['tmdbId'] = rating_dt['tmdbId'].astype(int)

In [ ]:
print(movie_dt['id'].dtype)
print(rating_dt['tmdbId'].dtype)

int64
int64


In [5]:
from scipy.sparse import csr_matrix

def create_user_item_matrix(rating_dt):
    # 사용자와 아이템(영화)의 고유 ID를 인덱스로 변환
    user_ids = rating_dt['userId']#.astype('category').cat.codes
    movie_ids = rating_dt['tmdbId']#.astype('category').cat.codes
    ratings = rating_dt['rating'].values

    # csr_matrix 생성
    user_item_matrix = csr_matrix((ratings, (user_ids, movie_ids)))

    return user_item_matrix, user_ids, movie_ids

# 사용자-아이템 행렬 생성
user_item_matrix, user_ids, movie_ids = create_user_item_matrix(rating_dt)

# 생성된 csr_matrix 정보 출력
print(f"User-Item Matrix shape: {user_item_matrix.shape}")
print(f"Number of users: {user_item_matrix.shape[0]}, Number of movies: {user_item_matrix.shape[1]}")


User-Item Matrix shape: (270897, 469173)
Number of users: 270897, Number of movies: 469173


In [6]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(
        factors=10,
        regularization=0.08,
        iterations=40,
        calculate_training_loss=False
    )
model.fit(user_item_matrix)

/Users/kimseohyun/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/kimseohyun/Library/Python/3.9/lib/python/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 40/40 [02:08<00:00,  3.21s/it]


In [9]:
from scipy.sparse import csr_matrix

def recommend_movie_by_als(user_id, n_movies=5, new_user_vector=None):
    # 사용자가 이미 평가한 영화와 평점 출력
    user_ratings = rating_dt[rating_dt['userId'] == user_id]
    print(f"User {user_id}의 평가 영화 및 평점:")
    for idx, row in user_ratings.iterrows():
        movie_title = movie_dt.loc[movie_dt['id'] == row['tmdbId'], 'title'].values
        if len(movie_title) > 0:
            print(f"- {movie_title[0]}: {row['rating']}점")
        else:
            print(f"- tmdbId {row['tmdbId']}에 해당하는 영화 제목을 찾을 수 없습니다.")

    # ALS 모델을 사용하여 추천 영화 목록 생성
    print(f"\nUser {user_id}에게 추천하는 영화 {n_movies}개:")
    if new_user_vector is None:
        recommendations = model.recommend(user_id, user_item_matrix[user_id], N=n_movies)
    else:
        # new_user_vector를 CSR 형식으로 변환
        new_user_vector_csr = csr_matrix(new_user_vector)

        # 변환된 CSR 형식을 사용하여 추천 생성, recalculate_user=True 사용
        recommendations = model.recommend(
            userid=user_id,
            user_items=new_user_vector_csr,
            N=n_movies,
            recalculate_user=True
        )

    # 추천된 영화 ID 목록 추출
    recommended_ids = recommendations[0]

    # 추천된 ID가 movie_dt에 있는지 확인 및 필터링
    valid_recommended_ids = [ids for ids in recommended_ids if ids in movie_dt['id'].values]
    missing_ids = [ids for ids in recommended_ids if ids not in movie_dt['id'].values]

    if missing_ids:
        print(f"movie_dt에서 찾을 수 없는 추천된 ID: {missing_ids}")
    else:
        print("모든 추천된 ID가 movie_dt에 있습니다.")

    # 추천된 영화 목록 출력
    for ids in valid_recommended_ids:
        movie_title = movie_dt.loc[movie_dt['id'] == ids, 'title'].values
        if len(movie_title) > 0:
            print(f"- {movie_title[0]}")
        else:
            print(f"- 추천된 id {ids}에 해당하는 영화 제목을 찾을 수 없습니다.")

    return


In [10]:
recommend_movie_by_als(100)

User 100의 평가 영화 및 평점:
- Heavenly Creatures: 3.0점
- Evil Dead II: 3.5점
- My Own Private Idaho: 3.5점

User 100에게 추천하는 영화 5개:
모든 추천된 ID가 movie_dt에 있습니다.
- The City of Lost Children
- Brazil
- Twelve Monkeys
- Trainspotting
- A Clockwork Orange


In [11]:
#예를들어, 이렇게 하자.
#영화별 평점을 10개만 물어보고, 해당 평점을 토대로 추천하고싶다.
import numpy as np

new_user_ratings = {
    'userId': rating_dt['userId'].max()+1,  
    'movieId': [25756,52106,36880,11959,12774,124124,267654,1412,24254,40448],
    'rating': [4, 5, 3, 4, 2, 5, 3, 4, 4, 5]  # 유저가 입력한 평점
    #평점부분 받을수있게 만들어야함
}

# DataFrame으로 변환
new_ratings_df = pd.DataFrame(new_user_ratings)

existing_movie_ids = user_item_matrix.shape[1]

# 신규 유저 평점 벡터 생성
new_user_vector = np.zeros(existing_movie_ids)

# 신규 유저의 평점을 벡터에 채워넣기
for _, row in new_ratings_df.iterrows():
    movie_index = row['movieId']  # 영화 ID
    new_user_vector[movie_index] = row['rating']  # 평점 입력

In [12]:
rating_dt['userId'].max()+1

np.int64(270897)

In [13]:
recommend_movie_by_als(rating_dt['userId'].max()+1,new_user_vector=new_user_vector)

User 270897의 평가 영화 및 평점:

User 270897에게 추천하는 영화 5개:
모든 추천된 ID가 movie_dt에 있습니다.
- Harold and Maude
- Manhattan
- Sleeper
- Annie Hall
- Some Like It Hot


In [15]:
import pickle
import os

def save_model(model, filepath='/Users/kimseohyun/Desktop/패스트캠퍼스/프로젝트/MLProject2_영화_추천_시스템/code/ml-project-ml-pjt-2/src/outputs/real_als_model.pkl'):
    # 저장할 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    
    # 모델 저장
    with open(filepath, 'wb') as file:
        pickle.dump(model, file)
    print(f"모델이 '{filepath}'에 저장되었습니다.")
    
save_model(model, filepath='/Users/kimseohyun/Desktop/패스트캠퍼스/프로젝트/MLProject2_영화_추천_시스템/code/ml-project-ml-pjt-2/src/outputs/real_als_model.pkl')

모델이 '/Users/kimseohyun/Desktop/패스트캠퍼스/프로젝트/MLProject2_영화_추천_시스템/code/ml-project-ml-pjt-2/src/outputs/real_als_model.pkl'에 저장되었습니다.
